In [1]:
import sys
sys.path.insert(0, 'C:/Users/nlab/Documents/GitHub/FreelyMovingEphys/')
import warnings
warnings.filterwarnings('ignore')


In [363]:
import json, os, cv2
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.cluster import KMeans
import matplotlib.colors as mcolors
import glob

from src.base import BaseInput
from src.topcam import Topcam
from src.utils.auxiliary import flatten_series, find_index_in_list
from src.utils.path import find


In [3]:
# metadata json path
metadata_path = 'C:/Users/nlab/Desktop/mike_bonsai/recordings/metadata/oa_metadata.json'
#C:\Users\nlab\Desktop\mike_bonsai\recordings\metadata
# task name
# 'oa' for object avoidance
# 'gd' for gap detection
task_name = 'oa'

In [229]:
## create avoidance session object 
# metadata is json file that specifys the path to recording session for preprocessing
class AvoidanceProcessing(BaseInput):
    def __init__(self, metadata_path, task='oa'):
        with open(metadata_path) as f:
            self.metadata = json.load(f)

        self.path = self.metadata['path']
        self.dates_list = [i for i in list(self.metadata.keys()) if i != 'path']
        if task=='oa':
            self.is_pillar_avoidance = True
            self.is_gap_detection = False
        #elif task=='gd':
            #self.is_pillar_avoidance = False
            #self.is_gap_detection = True

        if self.is_pillar_avoidance:
            self.dlc_project = {'light':'/home/niell_lab/Documents/deeplabcut_projects/object_avoidance-Mike-2021-08-31/config.yaml',
                                'dark':None}
        #elif self.is_gap_detection:
            #self.dlc_project = {'light':'/home/niell_lab/Documents/deeplabcut_projects/gap_determination-Kana-2021-10-19/config.yaml',
                                #'dark':'/home/niell_lab/Documents/deeplabcut_projects/dark_gap_determination-Kana-2021-11-08/config.yaml'}
        self.camname = 'top1'
        self.generic_camconfig = {
            'paths': {
                'dlc_projects': {
                    self.camname: self.dlc_project
                },
            },
            'internals': {
                'follow_strict_naming': False,
                'crop_for_dlc': False,
                'filter_dlc_predictions': False,
                'multianimal_top_project': False,
                'likelihood_threshold': 0.99
            }
        }
    ## gather sessions from jason metadata
    def gather_all_sessions(self):
        data_dict = {'date': [],
                    'animal': [],
                    'task': [],             
                    'poke1_ts':[],
                    'poke2_ts': [],
                    'top1_ts': [],
                    'poke1_t0':[],
                    'poke2_t0': [],
                    'top1_t0': []}
        # list of dates for analysis
        data_path = Path(self.path).expanduser()
        # populate dict with metadata and timestamps
        for date in self.dates_list:
            use_animals = [k for k,v in self.metadata[date].items()]
            for ani in use_animals:
                for task in os.listdir(data_path / date / ani):
                    data_paths = [str(i) for i in list((data_path / date / ani/ task).rglob('*.csv'))]
                    data_paths = [i for i in data_paths if 'spout' not in i]
                    if data_paths != []:
                        _, name = os.path.split(data_paths[1])
                        split_name = name.split('_')
                        data_dict['date'].append(split_name[0])
                        data_dict['animal'].append(split_name[1])
                        data_dict['task'].append(split_name[4])
                    for ind, csv in enumerate(data_paths):
                        self.timestamp_path = csv
                        time = self.read_timestamp_file()
                        _, name = os.path.split(data_paths[ind])
                        split_name = name.split('_')
                        data_dict[split_name[5] +'_ts'].append(time)
                        data_dict[split_name[5] +'_t0'].append(time[0])
        self.all_sessions = pd.DataFrame.from_dict(data_dict)
## process each session
    def process(self, videos=False):
        self.gather_all_sessions()
        for trial_ind, trial_row in tqdm(self.all_sessions.iterrows()):
            try:
                # analyze each trial
                trial = AvoidanceSession(trial_row, self.path, self.metadata)
                dlc_h5 = find('*'+str(trial_row['date'])+'*'+trial_row['animal']+'*'+str(trial_row['task'])+'*.h5', self.path)
                
                if dlc_h5 == []:
                    continue
                trial_path, _ = os.path.split(dlc_h5[0])
                trial_path = trial_path.replace(os.sep, '/')
                trial_name = '_'.join(os.path.splitext(os.path.split([i for i in find('*.avi', trial_path) if all(bad not in i for bad in ['plot','IR','rep11','betafpv','side_gaze','._'])][0])[1])[0].split('_')[:-1])
                trial.add_tracking(trial_name, trial_path)
                if self.is_pillar_avoidance:
                    trial.pillar_avoidance()
                elif self.is_gap_detection:
                    trial.gap_detection()
                # make short diagnostic video
                if videos:
                    trial.make_videos()
            except:
                pass
    
    


In [410]:
## Process individual recoring sessions

class AvoidanceSession(BaseInput):
    def __init__(self, s_input, path_input, metadata_input):
        self.s = s_input # series from dataframe of all trials
        self.likelihood_thresh = 0.99
        self.dist_across_arena = 30.48 # cm between bottom-right and bottom-left pillar
        self.path = path_input
        self.camname = 'top1'
        self.shared_metadata = metadata_input

        #self.num_clusters_to_use = self.shared_metadata[self.s['date']][self.s['animal']][str(self.s['task'])]['num_positions']
        self.session_path = os.path.join(*[self.path, str(self.s['date']), str(self.s['animal']),str(self.s['task'])])

        self.generic_camconfig = {
            'internals': {
                'follow_strict_naming': False,
                'likelihood_threshold': 0.99
            }
        }
    
    def make_task_df(self):
        #len of object is one 
        num_odd_trials = np.min([len(self.s['poke1_ts']), len(self.s['poke2_ts'])])
        df1 = pd.DataFrame([])
        count =  -1 
        print(num_odd_trials)
       
     
        
        for c in range(num_odd_trials):
            
            # odd
            count += 1
            df1.at[count, 'first_poke'] = self.s['poke1_ts'][c]
            df1.at[count, 'second_poke'] = self.s['poke2_ts'][c]
            time = self.s['top1_ts']; time = time[time > df1.loc[count,'first_poke']]; time = time[time < df1.loc[count,'second_poke']]
            df1.at[count, 'trial_timestamps'] = time.astype(object)
            start_stop_inds = (int(np.where([self.s['top1_ts']==time[0]])[1]), int(np.where([self.s['top1_ts']==time[-1]])[1]))
            for pos in list(self.positions['point_loc'].values):
                df1.at[count, pos] = np.array(self.positions.loc[start_stop_inds[0]:start_stop_inds[1], pos]).astype(object)
            df1.at[count, 'len'] = start_stop_inds[1] - start_stop_inds[0]
          
            # even
            count += 1
            if c+1 < len(self.s['poke1_ts']):
                df1.at[count, 'first_poke'] = self.s['poke2_ts'][c]
                df1.at[count, 'second_poke'] = self.s['poke1_ts'][c+1]
                time = self.s['top1_ts']; time = time[time > df1.loc[count,'first_poke']]; time = time[time < df1.loc[count,'second_poke']]
                vidframes = np.array(list(find_index_in_list(list(self.s['top1_ts']), list(time))))
                #df1.at[count, 'recording_timestamps'] = self.s['top1_ts'].astype(object)
                df1.at[count, 'trial_timestamps'] = time.astype(object)
                #df1.at[count, 'trial_vidframes'] = vidframes.astype(object)
                start_stop_inds = (int(np.where([self.s['top1_ts']==time[0]])[1]), int(np.where([self.s['top1_ts']==time[-1]])[1]))
                for pos in list(self.positions['point_loc'].values):
                    df1.at[count, pos] = np.array(self.positions.loc[start_stop_inds[0]:start_stop_inds[1], pos]).astype(object)
                df1.at[count, 'len'] = start_stop_inds[1] - start_stop_inds[0]
               
        df1['animal'] = self.s['animal']; df1['date'] = self.s['date']; df1['task'] = self.s['task']
        print('check')
        self.data = df1
        

        
    def add_tracking(self, name, path):
        tc = Topcam(self.generic_camconfig, name, path, self.camname)
        #print('hey')
        tc.gather_camera_files()
        tc.pack_position_data()
        tc.filter_likelihood()
        self.positions = tc.xrpts
        self.session_name = name
        self.session_path = path

    def convert_pxls_to_dist(self):
        x_cols = [i for i in self.data.columns.values if '_x' in i]
        y_cols = [i for i in self.data.columns.values if '_y' in i]
        for i in range(len(x_cols)):
            self.data[x_cols[i]+'_cm'] = self.data.loc[:,x_cols[i]] / self.pxls2cm
            self.data[y_cols[i]+'_cm'] = self.data.loc[:,y_cols[i]] / self.pxls2cm
    def get_median_trace(self,df):
        fake_time = np.linspace(0,1,100)
        all_nose_positions = np.zeros([len(df), 2, 100])
        count =  0
        for ind, row in df.iterrows():
            xT = np.linspace(0,1,len(row['nose_x_cm'])); yT = np.linspace(0,1,len(row['nose_y_cm']))
            all_nose_positions[count,0,:] = interp1d(xT, row['nose_x_cm'], bounds_error=False)(fake_time)
            all_nose_positions[count,1, :] = interp1d(yT, row['nose_y_cm'], bounds_error=False)(fake_time)
            count += 1
        median_trace = np.nanmedian(all_nose_positions, axis=0)
        for ind, row in df.iterrows():
            df.at[ind,'median_x_cm'] = median_trace[0,:].astype(object); df.at[ind,'median_y_cm'] = median_trace[1,:].astype(object)
    
    
    def pillar_avoidance(self):
        self.make_task_df()
        print('check')


        # label odd/even trials (i.e. moving leftwards or moving rightwards?)
        self.data['odd'] = np.nan
        for i, ind in enumerate(self.data.index.values):
            if ind%2 == 0: # odd values
                self.data.at[ind, 'odd'] = True
            elif ind%2 == 1:
                self.data.at[ind, 'odd'] = False
        ## convert pxl to cm 
        dist_to_posts = np.median(self.data['arenaTR_x'].iloc[0],0) - np.median(self.data['arenaTL_x'].iloc[0],0)
        self.pxls2cm = dist_to_posts/self.dist_across_arena
        self.convert_pxls_to_dist()


        # get obstacle position for all times
        for ind, row in self.data.iterrows():
            for x in ['b','w']:
                xvals = np.stack([row['obstacle'+x+'TL_x_cm'], row['obstacle'+x+'TR_x_cm'], row['obstacle'+x+'BL_x_cm'], row['obstacle'+x+'BR_x_cm']]).astype(float)
                xvals_cm = np.stack([row['obstacle'+x+'TL_x_cm'], row['obstacle'+x+'TR_x_cm'], row['obstacle'+x+'BL_x_cm'], row['obstacle'+x+'BR_x_cm']]).astype(float)
                self.data.at[ind, x+'obstacle_x'] = np.nanmean(xvals)
                self.data.at[ind, x+'obstacle_x_cm'] = np.nanmean(xvals_cm)
                self.data.at[ind, x+'obstacle_x_std'] = np.mean(np.nanstd(xvals, axis=1))
                yvals = np.stack([row['obstaclewTL_y_cm'], row['obstaclewTR_y_cm'], row['obstaclewBL_y_cm'], row['obstaclewBR_y_cm']]).astype(float)
                yvals_cm = np.stack([row['obstaclewTL_y_cm'], row['obstaclewTR_y_cm'], row['obstaclewBL_y_cm'], row['obstaclewBR_y_cm']]).astype(float)
                self.data.at[ind, x+'obstacle_y'] = np.nanmean(yvals)
                self.data.at[ind, x+'obstacle_y_cm'] = np.nanmean(yvals_cm)
                self.data.at[ind, x+'obstacle_y_std'] = np.mean(np.nanstd(yvals, axis=1))
        print('save')    
        print(self.session_path)   
       
        self.data.to_hdf(os.path.join(self.session_path, (self.data['animal'].iloc[0]+'_'+str(self.data['date'].iloc[0])+'_'+str(self.data['task'].iloc[0])+'.h5')), 'w')
    


In [412]:
pd.read_hdf('C:/Users/nlab/Desktop/mike_bonsai/recordings/111021/G6H313TT/oa/G6H313TT_111021_oa.h5')

,first_poke,second_poke,trial_timestamps,nose_x,nose_y,nose_likelihood,leftear_x,leftear_y,leftear_likelihood,rightear_x,...,bobstacle_x_std,bobstacle_y,bobstacle_y_cm,bobstacle_y_std,wobstacle_x,wobstacle_x_cm,wobstacle_x_std,wobstacle_y,wobstacle_y_cm,wobstacle_y_std
0,57638.425548,57644.635456,"[57638.42935, 57638.444825, 57638.461875, 5763...","[57.58335494995117, 57.5283203125, 57.66237258...","[210.811767578125, 211.5140380859375, 211.2993...","[0.9992797374725342, 0.9991064667701721, 0.999...","[76.29540252685547, 76.38246154785156, 76.8044...","[220.60549926757812, 221.0747528076172, 221.65...","[0.9993463158607483, 0.9994148015975952, 0.999...","[83.68081665039062, 83.17990112304688, 83.8800...",...,NaN,7.952389,7.952389,0.080488,21.607665,21.607665,0.052713,7.952389,7.952389,0.080488
1,57644.635456,57651.447756,"[57644.649228, 57644.664512, 57644.68151, 5764...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.5383057594299316, 0.4558617174625397, 0.410...","[482.2186584472656, 481.5039367675781, 483.961...","[187.2709503173828, 188.43711853027344, 192.22...","[0.9997715950012207, 0.9993929862976074, 0.998...","[476.45892333984375, 475.3243408203125, nan, n...",...,NaN,7.878984,7.878984,0.054816,21.576582,21.576582,0.049724,7.878984,7.878984,0.054816
2,57651.447756,57657.891276,"[57651.450124, 57651.466457, 57651.483635, 576...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.08736050128936768, 0.05651864409446716, 0.0...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.12561491131782532, 0.07208704948425293, 0.0...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,NaN,7.937462,7.937462,0.047927,21.596505,21.596505,0.038436,7.937462,7.937462,0.047927
3,57657.891276,57663.199808,"[57657.901388, 57657.917862, 57657.935168, 576...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.5633265972137451, 0.4086170792579651, 0.482...","[483.51031494140625, 483.28167724609375, 484.7...","[186.76124572753906, 188.24697875976562, 189.4...","[0.9997113943099976, 0.9992942810058594, 0.998...","[478.0167541503906, 477.82952880859375, 477.38...",...,NaN,7.923565,7.923565,0.040764,21.602495,21.602495,0.035588,7.923565,7.923565,0.040764
4,57663.199808,57668.943436,"[57663.203904, 57663.219968, 57663.23703, 5766...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.24882733821868896, 0.2872749865055084, 0.27...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.03773382306098938, 0.058313459157943726, 0....","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,NaN,10.176664,10.176664,2.644400,16.531830,16.531830,5.977835,10.176664,10.176664,2.644400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,58801.465203,58809.997772,"[58801.473907, 58801.490611, 58801.507328, 588...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.8870009779930115, 0.7979320883750916, 0.878...","[478.69677734375, 479.12255859375, 480.2549743...","[189.90383911132812, 189.62103271484375, 189.9...","[0.9983533620834351, 0.9978804588317871, 0.998...","[486.1778564453125, 486.9254150390625, 488.087...",...,NaN,8.490277,8.490277,0.118532,15.020697,15.020697,0.073306,8.490277,8.490277,0.118532
204,58809.997772,58814.716864,"[58810.010752, 58810.027379, 58810.044057, 588...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.8318108320236206, 0.7431681752204895, 0.739...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.959570050239563, 0.9377375841140747, 0.9169...","[nan, nan, nan, nan, nan, nan, 

In [413]:
pd.read_hdf('C:/Users/nlab/Desktop/mike_bonsai/recordings/111021/111021_output/G6H313TT_111021_oa.h5')

,first_poke,second_poke,trail_timestamps,nose_x,nose_y,nose_likelihood,leftear_x,leftear_y,leftear_likelihood,rightear_x,...,bobstacle_x_cm,bobstacle_x_std,bobstacle_y_cm,bobstacle_y_std,wobstacle_x_cm,wobstacle_x_std,wobstacle_y_cm,wobstacle_y_std,speed,obstacle_cluster
0,57638.425548,57644.635456,"[57638.42935, 57638.444825, 57638.461875, 5763...","[57.58335494995117, 57.5283203125, 57.66237258...","[210.811767578125, 211.5140380859375, 211.2993...","[0.9992797374725342, 0.9991064667701721, 0.999...","[76.29540252685547, 76.38246154785156, 76.8044...","[220.60549926757812, 221.0747528076172, 221.65...","[0.9993463158607483, 0.9994148015975952, 0.999...","[83.68081665039062, 83.17990112304688, 83.8800...",...,NaN,NaN,7.897998,0.080488,21.591793,0.052713,7.897998,0.080488,"[2.545177770572186, 0.830111009211383, 3.41651...",2
1,57644.635456,57651.447756,"[57644.649228, 57644.664512, 57644.68151, 5764...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.5383057594299316, 0.4558617174625397, 0.410...","[482.2186584472656, 481.5039367675781, 483.961...","[187.2709503173828, 188.43711853027344, 192.22...","[0.9997715950012207, 0.9993929862976074, 0.998...","[476.45892333984375, 475.3243408203125, nan, n...",...,NaN,NaN,7.974929,0.054816,21.652620,0.049724,7.974929,0.054816,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",2
2,57651.447756,57657.891276,"[57651.450124, 57651.466457, 57651.483635, 576...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.08736050128936768, 0.05651864409446716, 0.0...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.12561491131782532, 0.07208704948425293, 0.0...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,NaN,NaN,7.900225,0.047927,21.589726,0.038436,7.900225,0.047927,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",2
3,57657.891276,57663.199808,"[57657.901388, 57657.917862, 57657.935168, 576...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.5633265972137451, 0.4086170792579651, 0.482...","[483.51031494140625, 483.28167724609375, 484.7...","[186.76124572753906, 188.24697875976562, 189.4...","[0.9997113943099976, 0.9992942810058594, 0.998...","[478.0167541503906, 477.82952880859375, 477.38...",...,NaN,NaN,7.911500,0.040764,21.596866,0.035588,7.911500,0.040764,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",2
4,57663.199808,57668.943436,"[57663.203904, 57663.219968, 57663.23703, 5766...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.24882733821868896, 0.2872749865055084, 0.27...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.03773382306098938, 0.058313459157943726, 0....","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,NaN,NaN,7.920957,2.644400,21.584846,5.977835,7.920957,2.644400,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,58765.774028,58771.299942,"[58765.779468, 58765.795148, 58765.812224, 587...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.6548518538475037, 0.6947235465049744, 0.747...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.9426860809326172, 0.9091545939445496, 0.919...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,NaN,NaN,12.304940,0.032539,16.170756,0.037411,12.304940,0.032539,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",1
199,58771.299942,58776.719398,"[58771.314636, 58771.330892, 58771.348428, 587...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.9254875183105469, 0.9562804102897644, 0.950...","[476.57

In [383]:
session = AvoidanceProcessing(metadata_path, task=task_name)

In [385]:
session.gather_all_sessions()

In [ ]:
session.data

In [411]:
session.process()

0it [00:00, ?it/s]

['C:/Users/nlab/Desktop/mike_bonsai/recordings\\111021\\G6H313TT\\oa\\111021_G6H313TT_control_Rig2_oa_top1DLC_resnet50_object_avoidanceAug31shuffle1_200000.h5', 'C:/Users/nlab/Desktop/mike_bonsai/recordings\\prelim_data\\oa\\111021\\G6H313TT\\oa\\111021_G6H313TT_control_Rig2_oa_top1DLC_resnet50_object_avoidanceAug31shuffle1_200000.h5']
C:/Users/nlab/Desktop/mike_bonsai/recordings/111021/G6H313TT/oa
111021_G6H313TT_control_Rig2_oa
104
check
check
save
C:/Users/nlab/Desktop/mike_bonsai/recordings/111021/G6H313TT/oa


1it [00:28, 28.88s/it]


In [143]:
num_odd_trials = np.min([len(session.all_sessions['poke1_ts']), len(session.all_sessions['poke2_ts'])])

In [144]:
num_odd_trials

1

In [139]:
len(session.all_sessions['poke1_ts'])

1

In [386]:
test = session.all_sessions

In [164]:
test['poke1_ts'][0][c]

57651.447756

In [165]:
session.position

AttributeError: 'DataFrame' object has no attribute 'positions'

In [359]:
num = np.min([len(test['poke1_ts'][0]), len(test['poke2_ts'][0])])
df1 = pd.DataFrame([])
count =  -1 
for c in range(num):
            # odd
            count += 1
            df1.at[count, 'first_poke'] = test['poke1_ts'][0][c]
            df1.at[count, 'second_poke'] = test['poke2_ts'][0][c]
            time = test['top1_ts'][0]; time = time[time > df1.loc[count,'first_poke']]; time = time[time < df1.loc[count,'second_poke']]
            vidframes = np.array(list(find_index_in_list(list(test['top1_ts'][0]), list(time))))
            df1.at[count, 'recording_timestamps'] = test['top1_ts'][0].astype(object)
            df1.at[count, 'trial_timestamps'] = time.astype(object)
            df1.at[count, 'trial_vidframes'] = vidframes[0].astype(object)
            start_stop_inds = (int(np.where([test['top1_ts'][0]==time[0]])[1]), int(np.where([test['top1_ts'][0]==time[-1]])[1]))
            np.array(pos.xrpts.loc[start_stop_inds[0]:start_stop_inds[1], posi]).astype(object)
            for posi in list(pos.xrpts['point_loc'].values):
                df1.at[count, posi] = np.array(pos.xrpts.loc[start_stop_inds[0]:start_stop_inds[1], posi]).astype(object)
            df1.at[count, 'len'] = start_stop_inds[1] - start_stop_inds[0]
            # even
            count += 1
            if c+1 < len(test['poke1_ts'][0]):
                df1.at[count, 'first_poke'] = test['poke2_ts'][0][c]
                df1.at[count, 'second_poke'] = test['poke1_ts'][0][c+1]
                time = test['top1_ts'][0]; time = time[time > df1.loc[count,'first_poke']]; time = time[time < df1.loc[count,'second_poke']]
                vidframes = np.array(list(find_index_in_list(list(test['top1_ts'][0]), list(time))))
                df1.at[count, 'recording_timestamps'] = test['top1_ts'][0].astype(object)
                df1.at[count, 'trial_timestamps'] = time.astype(object)
                df1.at[count, 'trial_vidframes'] = vidframes[0].astype(object)
                start_stop_inds = (int(np.where([test['top1_ts'][0]==time[0]])[1]), int(np.where([test['top1_ts'][0]==time[-1]])[1]))
                for posi in list(pos.xrpts['point_loc'].values):
                    df1.at[count, posi] = np.array(pos.xrpts.loc[start_stop_inds[0]:start_stop_inds[1], posi]).astype(object)
                df1.at[count, 'len'] = start_stop_inds[1] - start_stop_inds[0]
            df1['animal'] = test['animal']; df1['date'] = test['date']; df1['task'] = test['task']
        
            


57638.42935
65
65
373
57644.649228
438
438
408
57651.450124
846
846
387
57657.901388
1233
1233
318
57663.203904
1551
1551
345
57668.955174
1896
1896
387
57675.407462
2283
2283
265
57679.825792
2548
2548
319
57685.145062
2867
2867
278
57689.779814
3145
3145
328
57695.248332
3473
3473
282
57699.949888
3755
3755
318
57705.250841
4073
4073
291
57710.1056
4364
4364
343
57715.82103
4707
4707
276
57720.423564
4983
4983
340
57726.092249
5323
5323
250
57730.259238
5573
5573
318
57735.562112
5891
5891
250
57739.729241
6141
6141
323
57745.115328
6464
6464
286
57749.884876
6750
6750
337
57755.502412
7087
7087
261
57759.85294
7348
7348
334
57765.421337
7682
7682
813
57778.976844
8495
8495
319
57784.294835
8814
8814
276
57788.896038
9090
9090
320
57794.232012
9410


IndexError: index 0 is out of bounds for axis 0 with size 0

In [360]:
df1[['recording_timestamps','trial_timestamps','trial_vidframes']]

,recording_timestamps,trial_timestamps,trial_vidframes
0,"[57637.345228, 57637.361088, 57637.377792, 576...","[57638.42935, 57638.444825, 57638.461875, 5763...","[65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 7..."
1,"[57637.345228, 57637.361088, 57637.377792, 576...","[57644.649228, 57644.664512, 57644.68151, 5764...","[438, 439, 440, 441, 442, 443, 444, 445, 446, ..."
2,"[57637.345228, 57637.361088, 57637.377792, 576...","[57651.450124, 57651.466457, 57651.483635, 576...","[846, 847, 848, 849, 850, 851, 852, 853, 854, ..."
3,"[57637.345228, 57637.361088, 57637.377792, 576...","[57657.901388, 57657.917862, 57657.935168, 576...","[1233, 1234, 1235, 1236, 1237, 1238, 1239, 124..."
4,"[57637.345228, 57637.361088, 57637.377792, 576...","[57663.203904, 57663.219968, 57663.23703, 5766...","[1551, 1552, 1553, 1554, 1555, 1556, 1557, 155..."
5,"[57637.345228, 57637.361088, 57637.377792, 576...","[57668.955174, 57668.972172, 57668.988518, 576...","[1896, 1897, 1898, 1899, 1900, 1901, 1902, 190..."
6,"[57637.345228, 57637.361088, 57637.377792, 576...","[57675.407462, 57675.424217, 57675.441203, 576...","[2283, 2284, 2285, 2286, 2287, 2288, 2289, 229..."
7,"[57637.345228, 57637.361088, 57637.377792, 576...","[57679.825792, 57679.843443, 57679.859635, 576...","[2548, 2549, 2550, 2551, 2552, 2553, 2554, 255..."
8,"[57637.345228, 57637.361088, 57637.377792, 576...","[57685.145062, 57685.160665, 57685.177318, 576...","[2867, 2868, 2869, 2870, 2871, 2872, 2873, 287..."
9,"[57637.345228, 57637.361088, 57637.377792, 576...","[57689.779814, 57689.797286, 57689.812569, 576...","[3145, 3146, 3147, 3148, 3149, 3150, 3151, 315..."


In [327]:
vidframes = np.array(list(set(list(test['top1_ts'][0])).intersection(list(time))))

In [335]:
vidframes


array([], dtype=float64)

In [336]:
count

28

In [353]:
def find_index_in_list(a, subset):
    """
    Parameters
    --------
    a : list
        list of values
    subset : list
        list of values shorter than a, which may exist in a
    
    Returns
    --------
    (idx, )
    """
    if not subset:
        return
    subset_len = len(subset)
    first_val = subset[0]
    for idx, item in enumerate(a):
        if item == first_val:
            print(item)
            print(idx)
            if a[idx:idx+subset_len] == subset:
                print(idx)
                print(subset_len)
                yield tuple(range(idx, idx+subset_len))

In [354]:
a = list(test['top1_ts'][0])
subset = list(time)
list(find_index_in_list(a, subset))


57794.232012
9410


[]

In [387]:
sum(np.isnan(test['top1_ts'][0]))

1

In [355]:
a[9410:9410+245]

[57794.232012,
 57794.248102,
 57794.264448,
 57794.281241,
 57794.29838,
 57794.31543,
 57794.333388,
 57794.34871,
 57794.365427,
 57794.382694,
 57794.398758,
 57794.415833,
 57794.432422,
 57794.4491,
 57794.466406,
 57794.482137,
 57794.498777,
 57794.516134,
 57794.532096,
 57794.549107,
 57794.565798,
 57794.582438,
 57794.599872,
 57794.614553,
 57794.631219,
 57794.648217,
 57794.664819,
 57794.682393,
 57794.699225,
 57794.715827,
 57794.733222,
 57794.748812,
 57794.76558,
 57794.783027,
 57794.798848,
 57794.815897,
 57794.83246,
 57794.84919,
 57794.866572,
 57794.882227,
 57794.898982,
 57794.916262,
 57794.932236,
 57794.949273,
 57794.965939,
 57794.982617,
 57794.999884,
 57795.01559,
 57795.03232,
 57795.049651,
 57795.065638,
 57795.082662,
 57795.099212,
 57795.11598,
 57795.13326,
 57795.149004,
 57795.165683,
 57795.184358,
 57795.199065,
 57795.215667,
 57795.233088,
 57795.248947,
 57795.265689,
 57795.283046,
 57795.29806,
 57795.314803,
 57795.331571,
 57795.3

In [344]:
subset

[57794.232012,
 57794.248102,
 57794.264448,
 57794.281241,
 57794.29838,
 57794.31543,
 57794.333388,
 57794.34871,
 57794.365427,
 57794.382694,
 57794.398758,
 57794.415833,
 57794.432422,
 57794.4491,
 57794.466406,
 57794.482137,
 57794.498777,
 57794.516134,
 57794.532096,
 57794.549107,
 57794.565798,
 57794.582438,
 57794.599872,
 57794.614553,
 57794.631219,
 57794.648217,
 57794.664819,
 57794.682393,
 57794.699225,
 57794.715827,
 57794.733222,
 57794.748812,
 57794.76558,
 57794.783027,
 57794.798848,
 57794.815897,
 57794.83246,
 57794.84919,
 57794.866572,
 57794.882227,
 57794.898982,
 57794.916262,
 57794.932236,
 57794.949273,
 57794.965939,
 57794.982617,
 57794.999884,
 57795.01559,
 57795.03232,
 57795.049651,
 57795.065638,
 57795.082662,
 57795.099212,
 57795.11598,
 57795.13326,
 57795.149004,
 57795.165683,
 57795.184358,
 57795.199065,
 57795.215667,
 57795.233088,
 57795.248947,
 57795.265689,
 57795.283046,
 57795.29806,
 57795.314803,
 57795.331571,
 57795.3

In [330]:
 b = list(set(list(test['top1_ts'][0])).intersection(list(time)))
 len(b)

245

In [306]:
vidframes

array([], dtype=float64)

In [303]:

len(list(time))

245

In [304]:

len(list(test['top1_ts'][0]))

73039

In [300]:
list(find_index_in_list(list(test['top1_ts'][0]), list(time)))


[]

In [186]:
pos.xrpts.loc[start_stop_inds[0]:start_stop_inds[1], posi]

<xarray.DataArray 'top1' (frame: 373)>
array([ 57.58335495,  57.52832031,  57.66237259,  58.19469833,
        57.3862114 ,  57.28472519,  59.04969025,  58.72245026,
        58.14268112,  57.91467667,  57.60771942,  57.3940773 ,
        57.37003326,  57.30064011,  57.29908752,  57.66080475,
        57.57054138,  58.32077789,  57.76386261,  57.74930191,
        58.02555084,  57.60464478,  57.76448059,  57.6974411 ,
        57.37197495,  57.78866196,  57.84939194,  57.53570175,
        57.36403656,  57.51089859,  57.18789673,  57.47966003,
        57.78370285,  57.54619217,  57.58495331,  57.60416031,
        57.3825264 ,  57.46607971,  57.91430283,  58.06296158,
        57.73070145,  57.30123901,  57.83438492,  57.61351776,
        57.50213242,  57.25500488,  56.97949982,  57.13237   ,
        56.98737335,  57.36495209,  57.71638107,  58.51764297,
        57.33112335,  57.00911713,  57.3027153 ,  58.01050949,
        57.2444725 ,  57.20707321,  57.01145935,  56.86823273,
        57.29023361,  57.49639893,  57.23594666,  57.24179459,
        57.43840408,  56.99274063,  58.21231461,  58.27040863,
        58.51312637,  57.05929184,  57.18435669,  57.49317169,
        58.35777664,  57.91558838,  57.54035568,  57.07418442,
        56.85573196,  57.64853668,  57.76146698,  58.08586121,
...
       504.93630981, 504.72213745, 504.94274902, 504.9699707 ,
       505.31011963, 505.70254517, 505.58062744, 504.57556152,
       504.85064697, 504.8956604 , 503.83917236, 505.17788696,
       505.72091675, 505.81661987, 506.35449219, 505.387146  ,
       505.31039429, 504.93008423, 505.65777588, 505.22909546,
       504.95852661, 507.60263062, 508.42730713, 507.27215576,
       507.1756897 , 506.40817261, 506.08868408, 506.04949951,
       506.28018188, 505.73486328, 507.30178833, 506.76385498,
       506.18704224, 504.74462891, 505.02355957, 505.8918457 ,
       507.50939941, 506.09085083, 505.95071411, 504.9392395 ,
       503.75585938, 503.65557861, 503.26556396, 503.78890991,
       503.17843628, 503.22125244, 503.2102356 , 503.6895752 ,
       503.93164062, 503.11294556, 503.6907959 , 503.98452759,
       503.59402466, 503.52310181, 502.98602295, 500.64645386,
       500.58828735, 500.76675415, 502.01602173, 502.82598877,
       503.1362915 , 503.22970581, 503.35739136, 504.54263306,
       504.21670532, 503.24966431, 503.33981323, 501.64837646,
       502.15200806, 501.64950562, 501.55862427, 500.43457031,
       501.41238403, 501.74081421, 502.70352173, 505.2097168 ,
       506.74758911])
Coordinates:
  * frame       (frame) int64 65 66 67 68 69 70 71 ... 432 433 434 435 436 437
    point_loc   <U6 'nose_x'
    timestamps  (frame) float64 5.764e+04 5.764e+04 ... 5.764e+04 5.764e+04

In [187]:
np.array(pos.xrpts.loc[start_stop_inds[0]:start_stop_inds[1], posi]).astype(object)

array([57.58335494995117, 57.5283203125, 57.66237258911133,
       58.194698333740234, 57.38621139526367, 57.284725189208984,
       59.04969024658203, 58.722450256347656, 58.14268112182617,
       57.914676666259766, 57.60771942138672, 57.39407730102539,
       57.370033264160156, 57.30064010620117, 57.29908752441406,
       57.660804748535156, 57.57054138183594, 58.320777893066406,
       57.76386260986328, 57.74930191040039, 58.025550842285156,
       57.604644775390625, 57.76448059082031, 57.69744110107422,
       57.37197494506836, 57.78866195678711, 57.84939193725586,
       57.535701751708984, 57.364036560058594, 57.51089859008789,
       57.187896728515625, 57.47966003417969, 57.7837028503418,
       57.54619216918945, 57.58495330810547, 57.60416030883789,
       57.38252639770508, 57.46607971191406, 57.914302825927734,
       58.06296157836914, 57.7307014465332, 57.301239013671875,
       57.83438491821289, 57.61351776123047, 57.502132415771484,
       57.2550048828125, 56.979

In [129]:
session.gather_all_sessions()

In [130]:
session.all_sessions

,date,animal,task,poke1_ts,poke2_ts,top1_ts,poke1_t0,poke2_t0,top1_t0
0,111021,G6H313TT,oa,"[57638.425548, 57651.447756, 57663.199808, 576...","[57644.635456, 57657.891276, 57668.943436, 576...","[57637.345228, 57637.361088, 57637.377792, 576...",57638.425548,57644.635456,57637.345228


In [17]:
session_name = '_'.join(os.path.splitext(os.path.split([i for i in find('*.avi', session_path[0]) if all(bad not in i for bad in ['plot','IR','rep11','betafpv','side_gaze','._'])][0])[1])[0].split('_')[:-1])

NameError: name 'session_path' is not defined

In [179]:
pos = Topcam(session.generic_camconfig,'111021_G6H313TT_control_Rig2_oa','C:/Users/nlab/Desktop/mike_bonsai/recordings/111021/G6H313TT/oa',session.camname)

In [180]:
pos.gather_camera_files()

In [181]:
pos.pack_position_data()

In [182]:
pos.xrpts['point_loc'].values

array(['nose_x', 'nose_y', 'nose_likelihood', 'leftear_x', 'leftear_y',
       'leftear_likelihood', 'rightear_x', 'rightear_y',
       'rightear_likelihood', 'spine_x', 'spine_y', 'spine_likelihood',
       'midspine_x', 'midspine_y', 'midspine_likelihood', 'tailbase_x',
       'tailbase_y', 'tailbase_likelihood', 'midtail_x', 'midtail_y',
       'midtail_likelihood', 'tailend_x', 'tailend_y',
       'tailend_likelihood', 'arenaTL_x', 'arenaTL_y',
       'arenaTL_likelihood', 'arenaTR_x', 'arenaTR_y',
       'arenaTR_likelihood', 'arenaBL_x', 'arenaBL_y',
       'arenaBL_likelihood', 'arenaBR_x', 'arenaBR_y',
       'arenaBR_likelihood', 'obstaclewTL_x', 'obstaclewTL_y',
       'obstaclewTL_likelihood', 'obstaclewTR_x', 'obstaclewTR_y',
       'obstaclewTR_likelihood', 'obstaclewBR_x', 'obstaclewBR_y',
       'obstaclewBR_likelihood', 'obstaclewBL_x', 'obstaclewBL_y',
       'obstaclewBL_likelihood', 'obstaclebTL_x', 'obstaclebTL_y',
       'obstaclebTL_likelihood', 'obstaclebTR_x', '

In [69]:
test.add_tracking()


AttributeError: 'Topcam' object has no attribute 'add_tracking'

In [30]:
 h5_paths = [x for x in find(('*.h5'), 'C:/Users/nlab/Desktop/mike_bonsai/recordings/111021/G6H313TT/oa') if x != []]

In [31]:
h5_paths

['C:/Users/nlab/Desktop/mike_bonsai/recordings/111021/G6H313TT/oa\\111021_G6H313TT_control_Rig2_oa_top1DLC_resnet50_object_avoidanceAug31shuffle1_200000.h5']

In [24]:
session.path

'C:/Users/nlab/Desktop/mike_bonsai/recordings'

In [ ]:
session_path = os.path.split(dlc_h5[0])

In [71]:
session.path

'C:/Users/nlab/Desktop/mike_bonsai/recordings'

In [72]:
test.recording_path

'C:/Users/nlab/Desktop/mike_bonsai/recordings/111021/G6H313TT/oa'

In [73]:
test.config

{'paths': {'dlc_projects': {'top1': {'light': '/home/niell_lab/Documents/deeplabcut_projects/object_avoidance-Mike-2021-08-31/config.yaml',
    'dark': None}}},
 'internals': {'follow_strict_naming': False,
  'crop_for_dlc': False,
  'filter_dlc_predictions': False,
  'multianimal_top_project': False,
  'likelihood_threshold': 0.99}}

In [78]:
test1 = AvoidanceSession(session.all_sessions, test.recording_path, session.metadata)

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'Series'

In [83]:
session.all_sessions['task']

0    oa
Name: task, dtype: object

In [87]:
os.path.join(*[session.path, str(session.all_sessions['date']), str(session.all_sessions['animal']),str(session.all_sessions['task'])])

'C:/Users/nlab/Desktop/mike_bonsai/recordings\\0    111021\nName: date, dtype: object\\0    G6H313TT\nName: animal, dtype: object\\0    oa\nName: task, dtype: object'

In [76]:
session.metadata[session.s['date']][session.s['animal']][str(session.s['task'])]['num_positions']

AttributeError: 'AvoidanceProcessing' object has no attribute 's'

In [88]:
str(session.all_sessions['task'])

'0    oa\nName: task, dtype: object'